In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit'

import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import csv
import tensorflow as tf
from tensorflow.keras.models import Model
from neurite.tf import models  # Assuming the module's location
import voxelmorph.tf.losses as vtml
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
import neurite as ne
import sys
import nibabel as nib
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from neurite_sandbox.tf.models import labels_to_labels
from neurite_sandbox.tf.utils.augment import add_outside_shapes
from neurite.tf.utils.augment import draw_perlin_full

import tensorflow.keras.layers as KL
import voxelmorph as vxm
from utils import *
import argparse
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pathlib
import surfa as sf
import re
import json

2024-02-23 11:46:31.321326: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 11:46:31.367928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/autofs/space/bal_004/users/jd1677/singularity-images/tensorflow_2.13.0-gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Issue loading cv2


In [2]:
feta = pathlib.Path('feta_resized_192')
feta_files = list(feta.glob('*.nii.gz'))
feta_label_maps = [np.uint8(f.dataobj) for f in map(nib.load, feta_files)]

In [27]:
def extract_slices(inp,stride):
    out = tf.transpose(inp[0], perm=(2, 0, 1, 3))
    return out#[:,:,stride]
    
with open("params_192.json", "r") as json_file:
    config = json.load(json_file)
num_shapes = 6

model4_config = config["labels_to_image_model_with_shapes"]
num_labels=9

model4_config["labels_out"] = {int(key): value for key, value in model4_config["labels_out"].items()}
in_shape=model4_config["in_shape"]
slice_stride = model4_config["slice_stride_min"]

labels_to_image_model_with_shapes = create_model(model4_config)

brain_maps = get_brain(feta_label_maps)
shapes = [draw_shapes(in_shape, num_labels) for _ in range(num_shapes)]

shapes = map(np.squeeze, shapes)
shapes = map(np.uint8, shapes)
shapes = [f + 7 + 1 for f in shapes]
gen = generator(brain_maps, shapes)
slice_stride=10
images = []
masks = []
for i in range(16):
    # Generate image from the generator
    im = next(gen)
    # print(im.shape)
    generated_img, y = labels_to_image_model_with_shapes(im)

    # Extract slices and squeeze the last axis
    generated_img = extract_slices(generated_img, slice_stride)
    y = extract_slices(y, slice_stride)
    print(generated_img.shape)
    generated_img = np.squeeze(generated_img, axis=-1)
    images.append(generated_img)
    # print(y.shape)
    # y= np.squeeze(np.squeeze(y, axis=0),axis=-1)
    # y= np.squeeze(y, axis=-1)
    masks.append(y)

num_row = 2
per_row = 8
for i in range(0, num_row * per_row, per_row):
    ne.plot.slices(images[i:i + per_row], cmaps=['gray'])
    # ne.plot.slices(masks[i:i + per_row], cmaps=['gray'])
    
# ne.plot.slices(generated_img)

# im = next(gen)
# generated_img, y = labels_to_image_model_with_shapes(im)

# generated_img= extract_slices(generated_img,slice_stride)
# generated_img = np.squeeze(generated_img, axis=-1)
# print(generated_img.shape)
# ne.plot.slices(generated_img)
# unet_model = vxm.networks.Unet(inshape=(*in_shape_3d, 1), nb_features=(en, de), 
#                                nb_conv_per_level=nb_conv_per_level,
#                                final_activation_function='softmax')
# input_img = Input(shape=(*in_shape_3d,1))

# if args.shapes:
#     generated_img, y = labels_to_image_model_with_shapes(input_img)

# generated_img= extract_slices(generated_img,slice_stride)

(None, 192, 192, 192, 1)
(None, 192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)
(192, 192, 192, 1)


AssertionError: each slice has to be 2d or RGB (3 channels)

In [17]:
print(masks[0].shape)

(192, 192, 192)
